In [1]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /opt/conda/envs/Python36/lib/python3.6/site-packages (20.0.2)


In [2]:
if not ('sc' in locals() or 'sc' in globals()):
    print('It seems you are not running in a IBM Watson Studio Apache Spark Notebook. You might be running in a IBM Watson Studio Default Runtime or outside IBM Waston Studio. Therefore installing local Apache Spark environment for you. Please do not use in Production')
    
    from pip import main
    main(['install', 'pyspark==2.4.5'])
    
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession

    sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
    
    spark = SparkSession \
        .builder \
        .getOrCreate()

It seems you are not running in a IBM Watson Studio Apache Spark Notebook. You might be running in a IBM Watson Studio Default Runtime or outside IBM Waston Studio. Therefore installing local Apache Spark environment for you. Please do not use in Production


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [3]:
!rm -f Student_Grades_Data*
!wget https://raw.githubusercontent.com/nitinkaushik01/Data_Science_Bootcamp/master/Regression_Algorithms/Simple_Linear_Regression/Student_Grades_Data.csv
df = spark.read.option("header", "true").option("inferSchema","true").csv('Student_Grades_Data.csv')
df.createOrReplaceTempView('df')
df.show()

--2020-04-03 18:10:41--  https://raw.githubusercontent.com/nitinkaushik01/Data_Science_Bootcamp/master/Regression_Algorithms/Simple_Linear_Regression/Student_Grades_Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328 [text/plain]
Saving to: ‘Student_Grades_Data.csv’

100%[======================================>] 328         --.-K/s   in 0s      

2020-04-03 18:10:41 (11.9 MB/s) - ‘Student_Grades_Data.csv’ saved [328/328]

+-------------+------+
|Time_to_Study|Grades|
+-------------+------+
|            1|   1.5|
|            5|   2.7|
|            7|   3.1|
|            3|   2.1|
|            2|   1.8|
|            9|   3.9|
|            6|   2.9|
|           12|   4.5|
|           11|   4.3|
|            2|   1.8|
|            4|   2.4|
|            8|   3.5|
|           13|   4.8|
|       

In [4]:
df.printSchema()

root
 |-- Time_to_Study: integer (nullable = true)
 |-- Grades: double (nullable = true)



In [5]:
from pyspark.ml.feature import VectorAssembler
feature_vec = VectorAssembler(inputCols=['Time_to_Study'], outputCol='features')

In [6]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="features", labelCol="Grades")

In [7]:
train_dataset, test_dataset = df.randomSplit([0.8,0.2])

In [8]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[feature_vec, lr])
model = pipeline.fit(train_dataset)
ppp = model.transform(test_dataset)

In [9]:
ppp.show()

+-------------+------+--------+------------------+
|Time_to_Study|Grades|features|        prediction|
+-------------+------+--------+------------------+
|            1|   1.5|   [1.0]| 1.554811057932708|
|            1|   1.5|   [1.0]| 1.554811057932708|
|            2|   1.8|   [2.0]|1.8285734695335365|
|            2|   1.8|   [2.0]|1.8285734695335365|
|            4|   2.4|   [4.0]| 2.376098292735194|
|            5|   2.7|   [5.0]|2.6498607043360227|
|            6|   2.9|   [6.0]|2.9236231159368513|
|            7|   3.1|   [7.0]|3.1973855275376803|
|            7|   3.1|   [7.0]|3.1973855275376803|
|            8|   3.5|   [8.0]|3.4711479391385094|
|            9|   3.9|   [9.0]| 3.744910350739338|
|           10|   4.1|  [10.0]| 4.018672762340167|
|           11|   4.3|  [11.0]| 4.292435173940995|
|           11|   4.3|  [11.0]| 4.292435173940995|
|           13|   4.8|  [13.0]| 4.839959997142653|
|           14|   5.0|  [14.0]| 5.113722408743482|
|           14|   5.0|  [14.0]|

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="Grades", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(ppp)
print("RMSE on test data = ", rmse)

RMSE on test data =  0.07537371270999911
